In [1]:
import re
from bs4 import BeautifulSoup
from gensim.models import KeyedVectors, Word2Vec  

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
RUSSIAN = re.compile(r"[А-ЯЁа-яё \-]+")

In [3]:
vectors = KeyedVectors.load_word2vec_format("./skipgram_vec.gz")

In [4]:
# загружаем список фразеологизмов, собираем подходящие, записываем

In [5]:
phrases_page = ""
with open("./rus_phrases.html") as f:
    for line in f.readlines():
        phrases_page += line + "\n"

In [6]:
soup = BeautifulSoup(phrases_page)

In [7]:
found_parts = soup.find_all("span", class_ = "html-attribute-value")

In [8]:
phrases = []
for part in found_parts:
    part = part.get_text().replace("(страница не существует)", "")
    if re.fullmatch(RUSSIAN, part.strip()):
        if len(part.split()) in range(2,4):
            phrases.append(part)

In [9]:
len(phrases)

904

In [10]:
# предобработка и собственно генерация названия

In [11]:
def is_noun_nominative(word):
    if ("NOUN" in morph.parse(word)[0].tag) and (morph.parse(word)[0].normal_form == word):
        return True
    return

In [12]:
def get_gender(word): 
    p = morph.parse(word)[0]
    return p.tag.gender

In [13]:
def replace_word(noun):
    closest_phrase = "синий мопс счастья" # это всамделишное название книги Донцовой
    new_phrases = [x.split() for x in phrases]
    closest_distance = 0
    closest_word = "PAD" 
    for i, phrase in enumerate(new_phrases):
        for word in phrase:
            if is_noun_nominative(word): 
                try:
                    distance = vectors.similarity(word+"_NOUN", noun+"_NOUN")
                    if (distance > closest_distance) and word != noun and \
                    get_gender(word) == get_gender(noun): 
                        closest_phrase = phrases[i]
                        closest_word = word
                        closest_distance = distance 
                except KeyError:
                    continue  
                    
    return closest_phrase.replace(closest_word, noun).strip()

In [14]:
replace_word("продюсер")

'отставной козы продюсер'

In [15]:
replace_word("лама")

'дойная лама'

In [16]:
replace_word("сникерс")

'кнут и сникерс'

In [17]:
replace_word("телепузик")

'телепузик на побегушках'

In [18]:
replace_word("мартышка")

'газетная мартышка'